In [26]:
from tensorflow import keras # broken for keras >= 2.0, use 1.2.2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, LeakyReLU
#from tensorflow.keras.activations import LeakyReLU
from tensorflow.keras.layers import Flatten, Dense, Activation, Reshape

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Input
from tensorflow.keras import Model
import numpy as np
import cv2


In [27]:
loss_pred_model='tiny_vgg'

if loss_pred_model=='tiny_vgg':
    model = Sequential()
    model.add(Convolution2D(16, (3, 3),input_shape=(224,224,3),padding='same',strides=(1,1)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Convolution2D(32,(3,3) ,padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='valid'))
    model.add(Convolution2D(64,(3,3) ,padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='valid'))
    model.add(Convolution2D(128,(3,3) ,padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='valid'))
    model.add(Convolution2D(256,(3,3) ,padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='valid'))
    model.add(Convolution2D(512,(3,3) ,padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='valid'))
    model.add(Convolution2D(1024,(3,3) ,padding='same'))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(1))



elif loss_pred_model=='EfficientNet':
    IMG_SIZE=224
    NUM_CLASSES=1
    inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    outputs = EfficientNetB0(include_top=True, weights=None, classes=NUM_CLASSES)(inputs)

    model = Model(inputs, outputs)
model.compile(optimizer="adam", loss="mse", metrics=["mse"])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 224, 224, 16)      448       
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 224, 224, 16)      0         
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 112, 112, 32)      4640      
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 112, 112, 32)      0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 56, 56, 64)       

In [28]:
x=np.load('x.npy')
y=np.load('y.npy')

batch = np.array([(cv2.resize(x[idx,0,...],(224,224))) 
                  for idx in range(x.shape[0])])
batch = 2*(batch/255.) - 1

y[np.isnan(y)]=0

In [29]:
hist = model.fit(batch,y, epochs=8, batch_size=16, verbose=True)
print(hist)

Epoch 1/8
7/7 [==============================] - 3s 294ms/step - loss: 3.8391 - mse: 3.8391
Epoch 2/8
7/7 [==============================] - 2s 298ms/step - loss: 0.0561 - mse: 0.0561
Epoch 3/8
7/7 [==============================] - 2s 301ms/step - loss: 0.0748 - mse: 0.0748
Epoch 4/8
7/7 [==============================] - 2s 297ms/step - loss: 0.0346 - mse: 0.0346
Epoch 5/8
7/7 [==============================] - 2s 293ms/step - loss: 0.0334 - mse: 0.0334
Epoch 6/8
7/7 [==============================] - 2s 303ms/step - loss: 0.0476 - mse: 0.0476
Epoch 7/8
7/7 [==============================] - 2s 290ms/step - loss: 0.0507 - mse: 0.0507
Epoch 8/8
7/7 [==============================] - 2s 292ms/step - loss: 0.0559 - mse: 0.0559


Writing model to disk

In [30]:
import tensorflowjs as tfjs

tfjs.converters.save_keras_model(model, './saved_model')